# Data visualization: homework 2
By: Khrystyna Kubatska

## Task:
Ваше завдання створити візуалізації, котрі відповідають на наступні питання:

- Як змінювалась структура генерації електроенергії за роками?

- Як залежить споживання електроенергії від дня року та години доби?

- Як змінюється генерація електроенергії з різних джерел впродовж доби?

- Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

- Як змінюється споживання електроенергії впродовж тижня?

Ви самостійно маєте обрати спосіб візуалізації. Ви також маєте написати короткий супровідний текст до кожної візуалізації, котрий пояснює, чому ви обрали саме цей спосіб презентації даних, які ще альтернативи ви розглядали, та чому зупинились саме на цьому варіанті (які його переваги та недоліки).

Джерело даних: Національна енергетична компанія "Укренерго"

Посилання на набір даних:  Погодинний баланс потужності ОЕС України. Історичні дані 2014-2020рр.

## Solution

In [1]:
import pandas as pd
import altair as alt
import datetime

In [2]:
df = pd.read_csv('2014-2020.csv')
df = df.drop(columns=['Unnamed: 12'])
df.head()

,Час/Дата,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD
0,24-31.12.2020,9235,2039,621,4942,385.0,0,16693,-405.0,-84.0,-11.0,-29.0
1,23-31.12.2020,9221,2159,707,5549,470.0,0,17805,0.0,-43.0,-212.0,-46.0
2,22-31.12.2020,9249,2377,709,5906,1000.0,0,18870,0.0,-13.0,-328.0,-30.0
3,21-31.12.2020,9256,2499,702,6329,909.0,322,19887,0.0,0.0,-64.0,-66.0
4,20-31.12.2020,9213,2521,702,6640,823.0,602,20387,0.0,-29.0,-48.0,-37.0


In [3]:
df['Hour'] = [int(i.split('-')[0]) for i in df['Час/Дата']]
df['Date'] = [i.split('-')[1] for i in df['Час/Дата']]

df['Day'] = [i.split('.')[0] for i in df['Date']]
df['Month'] = [i.split('.')[1] for i in df['Date']]
df['Year'] = [i.split('.')[2] for i in df['Date']]

df['Date2'] = pd.to_datetime(df['Date'])
df['Month-Day'] = df['Date2'].dt.strftime('%j')

seasons = ['Winter', 'Spring', 'Summer', 'Autumn']
df['Season'] = [seasons[int(i) // 3 % 4] for i in df['Month']]

weekdays = ['0:Monday', '1:Tuesday', '2:Wednesday', '3:Thursday', '4:Friday', '5:Saturday', '6:Sunday']
df['Weekday_n'] = df['Date2'].dt.weekday # Monday=0, Sunday=6
df['Weekday'] = [weekdays[i] for i in df['Weekday_n']]

df = df.drop(columns=['Weekday_n'])
df.head()

,Час/Дата,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,...,UK_MLD,Hour,Date,Day,Month,Year,Date2,Month-Day,Season,Weekday
0,24-31.12.2020,9235,2039,621,4942,385.0,0,16693,-405.0,-84.0,...,-29.0,24,31.12.2020,31,12,2020,2020-12-31,366,Winter,3:Thursday
1,23-31.12.2020,9221,2159,707,5549,470.0,0,17805,0.0,-43.0,...,-46.0,23,31.12.2020,31,12,2020,2020-12-31,366,Winter,3:Thursday
2,22-31.12.2020,9249,2377,709,5906,1000.0,0,18870,0.0,-13.0,...,-30.0,22,31.12.2020,31,12,2020,2020-12-31,366,Winter,3:Thursday
3,21-31.12.2020,9256,2499,702,6329,909.0,322,19887,0.0,0.0,...,-66.0,21,31.12.2020,31,12,2020,2020-12-31,366,Winter,3:Thursday
4,20-31.12.2020,9213,2521,702,6640,823.0,602,20387,0.0,-29.0,...,-37.0,20,31.12.2020,31,12,2020,2020-12-31,366,Winter,3:Thursday


## 1. Як змінювалась структура генерації електроенергії за роками?

Щоб відповісти на це питання, я використала Bar chart, де стовпці відповідають за роки, а значення по осі Y вказують на сумарну кількість згенерованої енергії. Кожен стовпець поділений на частини, які відображають кількість згерерованої енергії кожним джерелом.   
  
Однією з переваг такої візуалізації є відображення розподілу та загальної кількості згенерованої енергії. Недоліком цього способу візуалізації є незручність відслідковування динамічності/кількості по змінах кожного окремого джерела, тому я додала підказки (tooltips).  
  
Альтернативним варіантом є Line Chart, завдяки йому було б зручніше оцінювати динаміку змін та кількість електроенергії за джерелами, але також було б важче відслідковувати розподіл між ними.

In [4]:
df_sum_by_year = df.groupby('Year').sum().reset_index()


chart_1 = alt.Chart(df_sum_by_year).transform_fold(
                ['AES', 'TEC', 'VDE', 'TES', 'GES'],
                as_=['Source type', 'Quantity']
            ).mark_bar().encode(
                x = alt.X('Year:N', title='Year'),
                y = alt.Y('Quantity:Q', title='Quantity, MW'),
                color = alt.Color('Source type:N'),
                tooltip = alt.Tooltip(['Source type:N', 'Quantity:Q', 'Year:N'])
            ).configure_legend(
                titleFontSize=15,
                labelFontSize=14
            ).configure_axis(
                titleFontSize=15
            ).configure_title(
                fontSize=20             
            ).properties(width = 750, height = 400, 
                title='Structure of electricity generation over the years')

chart_1

alt.Chart(...)

## 2. Як залежить споживання електроенергії від дня року та години доби?

Для цієї візуалізції я використала HeatMap, де значення по осі X відображають дату, по осі Y - годину доби, а колір - кількість спожитої електроенергії. Також додана опція Tooltip. Перевагою цього графіка є компактність та тривимірність. Недоліком є велика завантаженість по датах та використання кольору (відтінків зокрема) як кількість спожитої електроенергії.

Одним з варіантів, які я розглядала, було показати на HeatMap усереднені дані за рік, але навіть у такому випадку, на осі Y б знаходилося 365-366 значень, що все рівно є доволі багато для порівняння.    
Альтернативним варіантом може бути використання двох окремих Line Graph для відображення усереднених змін споживання електроенергії взалежності від години доби та в залежності від дня року окремо. У цих візуалізаціях значення були б більш читабельнbvb, але важко було б прослідкувати одночасно залежність від дня року та години доби.

In [10]:
alt.data_transformers.enable('csv')

# df2_day = df.groupby(["Month-Day", "Hour"]).mean().round().reset_index()

DataTransformerRegistry.enable('csv')

In [11]:
chart2 = alt.Chart(df).mark_rect(
            ).encode(
                x=alt.X('Date:T', title='Date', axis=alt.Axis(format='%d.%m.%Y')),
                y=alt.Y('Hour:Q', title='Hour'),
                color=alt.Color('CONSUMPTION:Q', title='Consumption, MW'),
                tooltip=alt.Tooltip(['Date:T', 'Hour:Q', 'Consumption:Q']),
            ).configure_legend(
                titleFontSize=15,
                labelFontSize=14
            ).configure_axis(
                titleFontSize=15
            ).configure_title(
                fontSize=20             
            ).properties(width = 750, height = 400, 
                title='Electricity consumption during the day of the year and the time of the day')

chart2

alt.Chart(...)

## 3. Як змінюється генерація електроенергії з різних джерел впродовж доби?

Щоб відповісти на це питання, для візуалізації я використала Line Chart. Кожна лінія відповідає за конкретне джерело: значення по осі X відображають час доби, а значення по осі Y - кількість згенерованої цим джерелом енергії. Перевагою цієї візуалізації є зручність відслідковування динамічності змін кількості електроенергії.  
  
Альтернативним варіантом було використання Bar Chart. Опис переваг/недоліків зазначений у першому завданні.

In [5]:
df_day_time = df.groupby('Hour').mean().reset_index()


chart_3 = alt.Chart(df_day_time).transform_fold(
                    ['AES', 'TEC', 'VDE', 'TES', 'GES'],
                    as_=['Source type', 'Quantity']
                ).transform_loess('Hour', 'Quantity', groupby=['Source type']
                ).mark_line().encode(
                    x = alt.X('Hour:Q', title='Hour', sort='ascending', scale = alt.Scale(zero = False)),
                    y = alt.Y('Quantity:Q', scale = alt.Scale(zero = False), title='Quantity, MW'),
                    color = alt.Color('Source type:N'),
                    tooltip = alt.Tooltip(['Source type:N', 'Hour:Q', 'Quantity:Q'])
                ).configure_legend(
                    titleFontSize=15,
                    labelFontSize=14
                ).configure_axis(
                    titleFontSize=15
                ).configure_title(
                    fontSize=20     
                ).properties(width = 750, height = 400, 
                    title = 'Generation of electricity from different sources changing during a day')

chart_3

alt.Chart(...)

## 4. Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

Щоб відповісти на це питання, я зобразила 2 окремих HeatMaps. 
У першому графіку значенння по осі X відповідають за місяці року, значення по осі Y - за години доби, а колір відображає усереднені значення споживання електроенергії.  
Другий графік подібний, але значенння по осях Х та Y поміняні місцями (X - години, Y - пори року).
Недоліки такого варіанту - важкість оцінювання по кольору. В обох візуалізаціях для зручності додані tooltips.
  
Альтернативними варіантами було використати Line Charts, але, в такому випадку, лінії, що зображали б кількість споживання по місяцях (12), могли б перетинатися/накладатися, ускладнюючи оцінювання. Хоча, на мою думку, такий спосіб візуалізації краще б передавав динамічність.

In [6]:
df4_month = df.groupby(["Month", "Hour"]).mean().round().reset_index()
df4_season = df.groupby(["Season", "Hour"]).mean().round().reset_index()

In [7]:
chart_4_1 = alt.Chart(df4_month
                ).mark_rect().encode(
                    alt.Y('Hour:N', title='Hour'),
                    alt.X('Month:N', title='Month', axis=alt.Axis(labelAngle=0)),
                    color='CONSUMPTION:Q',
                    tooltip=['Month:N', 'Hour:N', 'CONSUMPTION:Q']
                ).configure_legend(
                    titleFontSize=15,
                    labelFontSize=14
                ).configure_axis(
                    titleFontSize=15
                ).configure_title(
                    fontSize=20             
                ).properties(width = 650, height = 350, 
                     title='Electricity consumption changing during the day in terms of months')

chart_4_1

alt.Chart(...)

In [8]:
chart_4_2 = alt.Chart(df4_season
                ).mark_rect().encode(
                    alt.X('Hour:N', title='Hour', axis=alt.Axis(labelAngle=0)),
                    alt.Y('Season:N', title='Season'),
                    color='CONSUMPTION:Q',
                    tooltip=['Season:N', 'Hour:N', 'CONSUMPTION:Q']
                ).configure_legend(
                    titleFontSize=15,
                    labelFontSize=14
                ).configure_axis(
                    titleFontSize=15
                ).configure_title(
                    fontSize=20             
                ).properties(width = 650, height = 300, 
                     title='Electricity consumption changing during the day in terms of seasons')

chart_4_2

alt.Chart(...)

## 5. Як змінюється споживання електроенергії впродовж тижня?

Щоб відповісти на це питання, я використала Line Chart. Вісь X відповідає за дні тижня, вісь Y - за кількість спожитої електроенергії.   

Альтернативою міг би бути Bar Chart (або використання кількох стовпчиків, подібно до візуалізації до питання 1, або використання одного стовпчика, який би складався з 7 частин, відповідно до днів тижня). 
  
Проте, на мою думку, значення на Line Chart є легшими для читання та такий спосіб візуалізації краще передає динамічність.

In [9]:
df5_weekday = df.groupby(["Weekday"]).mean().round().reset_index()


chart_5 = alt.Chart(df5_weekday).mark_line( 
                    point=alt.OverlayMarkDef(color="blue")
                ).encode(
                    alt.X('Weekday:N', title='Weekday', axis=alt.Axis(labelAngle=0)),
                    alt.Y('CONSUMPTION:Q', title='CONSUMPTION, MW', scale = alt.Scale(zero = False)),
                    tooltip=['Weekday:N', 'CONSUMPTION:Q']
                ).configure_legend(
                    titleFontSize=15,
                    labelFontSize=14
                ).configure_axis(
                    titleFontSize=15
                ).configure_title(
                    fontSize=20             
                ).properties(width = 750, height = 400, 
                     title='Electricity consumption during week')

chart_5

alt.Chart(...)